In [ ]:
import pandas as pd
import altair as alt

In [ ]:
file = '../Data/filtered_ppj_data/VAMPseq/G6PD_scores_consequence.csv'
alt.data_transformers.disable_max_rows()

In [ ]:
def read_data(file):
    df = pd.read_csv(file)

    df.loc[df['Mut'] == 'Stop', 'Mut'] = '*'
    df = df.dropna(subset = ['Mut'])

    return df

In [ ]:
def map(df):

    protein_length = 515
    order = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', '*']

    map = alt.Chart(df).mark_rect().encode(
        x = alt.X('position:Q',
                  title = 'Amino Acid Position',
                  bin = alt.Bin(maxbins = protein_length + 1),
                  axis = alt.Axis(values = list(range(0, 530, 50)))
                 ),
        y = alt.Y('Mut',
                  title = '',
                  sort = order
                 ),
        color = alt.Color('average_score',
                          title = 'VAMP-seq Score',
                          scale = alt.Scale(
                              scheme = 'bluepurple',
                              domain = [0, 1],
                              clamp = True,
                              reverse = True
                          )
                         )
    ).properties(
        width = 1750,
        height = 600
    )

    map.display()

    #map.save('/Users/ivan/Desktop/pillar_project_figs/20250918_G6PD_Heatmap.png', dpi = 400)

In [ ]:
def main():
    df = read_data(file)
    map(df)

In [ ]:
main()